<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=def4a3a5229427b0a60ec92b73b61ddce272f9bbe01b9ce5e417ca488fd4cb15
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-14 13:46:30
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -29.35 K (-0.21%)
Current PnL: -28.74 L (-18.33%)
CY Booked + Current PnL: -14.45 L (-9.21%)
-------------------
Total profit:  1.62 L
Total loss:  -30.36 L
-------------------
Total Booked + Current PnL: 12.53 L (9.65%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.19 L (67.12%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.83,15.29,5.30,21.40,16218.0,40576.0,306002.0,6.96,60.0,X-LC,6.35,14.0,2.50,2.18,22.58,XY25,NTT,FMCG
17,COALINDIA,484.83,1.38,6.92,11.51,19.23,18517.0,10416.0,160876.0,25.73,71.0,L-LC,8.68,182.0,0.56,1.15,29.67,XY25,ATH,MINING
77,TTKPRESTIG,770.00,0.63,-22.23,28.70,0.09,22494.0,-22400.0,78377.0,81.86,36.0,M-SC,3.68,253.0,-1.00,0.56,2.97,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.89,-14.76,17.33,0.02,25787.0,-25759.0,148800.0,127.12,52.0,M-SC,8.79,240.0,-1.00,1.06,29.43,OX40N,NTT,PAINTS
49,MASFIN,398.61,-0.99,-5.33,28.92,22.05,26826.0,-5220.0,92760.0,-18.25,44.0,H-SC,5.56,168.0,-0.19,0.66,34.83,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.0,3.88,-30.17,90.31,32.89,48689.0,-23294.0,53913.0,-765.75,73.0,L-MC,8.62,259.0,-0.48,0.38,48.47,XR,NTT,BANKS
74,TITAGARH,1548.0,3.41,-22.89,89.00,45.74,166201.0,-55434.0,186743.0,2.71,46.0,H-SC,3.02,175.0,-0.33,1.33,21.46,XY24,NTT,ENGINEERING
72,TATAELXSI,9161.0,1.75,-16.51,58.14,32.03,63993.0,-21768.0,110067.0,-9.66,67.0,H-SC,6.69,157.0,-0.34,0.78,23.28,OX40N,NTT,IT
15,CAMS,950.0,1.65,-5.30,31.44,24.48,74987.0,-13348.0,238508.0,-81.98,41.0,X-SC,4.00,86.0,-0.18,1.70,16.61,X40N,NTT,MISC
47,KANSAINER,340.0,1.61,-21.24,44.07,13.47,93605.0,-57267.0,212400.0,-67.53,53.0,H-SC,4.08,159.0,-0.61,1.51,8.00,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-2.55,-45.15,300.52,119.69,446323.0,-122244.0,148517.0,-44.56,29.0,H-SC,11.73,150.0,-0.27,1.06,17.33,AR,ATH,IT
44,JCHAC,2282.00,-2.33,-37.18,59.17,-0.01,47506.0,-47518.0,80287.0,16334.78,40.0,M-SC,13.40,235.0,-1.00,0.57,2.76,OX40N,BTT,AC
73,TCS,4389.97,-2.22,-12.14,37.39,20.71,124251.0,-45926.0,332311.0,-24.27,46.0,X-LC,2.12,3.0,-0.37,2.37,11.04,X40,ATH,IT
13,BSOFT,831.70,-2.09,-24.54,99.83,50.79,105951.0,-34522.0,106131.0,-3.36,46.0,H-SC,10.13,171.0,-0.33,0.76,23.57,XR,ATH,IT
33,HINDUNILVR,2922.00,-1.90,-8.81,24.65,13.67,57782.0,-22653.0,234410.0,-15.61,48.0,X-LC,3.10,9.0,-0.39,1.67,10.66,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.93,0.24,20.44,20.74,41629.0,495.0,203665.0,31.15,56.0,M-MC,5.23,189.0,0.01,1.45,10.88,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.42,-2.13,112.44,107.91,162470.0,-3149.0,144495.0,-20.55,52.0,M-SC,10.09,220.0,-0.02,1.03,7.74,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.89,-14.76,17.33,0.02,25787.0,-25759.0,148800.0,127.12,52.0,M-SC,8.79,240.0,-1.00,1.06,29.43,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,1.75,-16.51,58.14,32.03,63993.0,-21768.0,110067.0,-9.66,67.0,H-SC,6.69,157.0,-0.34,0.78,23.28,OX40N,NTT,IT
47,KANSAINER,340.00,1.61,-21.24,44.07,13.47,93605.0,-57267.0,212400.0,-67.53,53.0,H-SC,4.08,159.0,-0.61,1.51,8.00,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,0.63,-22.23,28.70,0.09,22494.0,-22400.0,78377.0,81.86,36.0,M-SC,3.68,253.0,-1.00,0.56,2.97,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.93,0.24,20.44,20.74,41629.0,495.0,203665.0,31.15,56.0,M-MC,5.23,189.0,0.01,1.45,10.88,XY25,NTT,FINANCE
17,COALINDIA,484.83,1.38,6.92,11.51,19.23,18517.0,10416.0,160876.0,25.73,71.0,L-LC,8.68,182.0,0.56,1.15,29.67,XY25,ATH,MINING
84,WIPRO,420.00,-1.59,6.64,61.54,72.27,111203.0,11252.0,180700.0,-8.35,47.0,M-LC,6.27,101.0,0.10,1.29,13.23,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.59,6.64,61.54,72.27,111203.0,11252.0,180700.0,-8.35,47.0,M-LC,6.27,101.0,0.10,1.29,13.23,XR,NTT,IT
25,FINCABLES,1641.55,1.42,-2.13,112.44,107.91,162470.0,-3149.0,144495.0,-20.55,52.0,M-SC,10.09,220.0,-0.02,1.03,7.74,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.74,-8.94,127.02,106.72,142652.0,-11029.0,112307.0,-54.82,32.0,H-SC,8.05,138.0,-0.08,0.80,12.35,AR,ATH,MISC
37,IEX,219.00,-1.36,-5.40,57.11,48.63,108025.0,-10802.0,189152.0,-36.95,48.0,H-SC,13.21,137.0,-0.10,1.35,5.59,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-1.78,-6.71,45.38,35.63,88727.0,-14055.0,195519.0,-19.84,33.0,H-MC,3.58,119.0,-0.16,1.39,8.85,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.09,-18.04,35.17,10.78,69272.0,-43356.0,196962.0,-51.37,13.0,X-LC,8.48,1.0,-0.63,1.40,0.0,X40,NTT,FMCG
7,AWL,485.00,-0.48,-29.11,121.73,57.19,281437.0,-94929.0,231198.0,-66.44,21.0,X-MC,10.46,58.0,-0.34,1.65,0.0,XY24,NTT,FMCG
22,DIXON,18931.72,-0.43,-22.42,69.18,31.26,123860.0,-51728.0,179040.0,-63.74,26.0,X-MC,20.76,56.0,-0.42,1.28,0.0,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.30,-41.61,131.55,35.20,98836.0,-53538.0,75132.0,-1.52,26.0,X-SC,17.59,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
52,PAGEIND,51605.07,-1.14,-13.31,50.67,30.62,104127.0,-31554.0,205500.0,-38.32,29.0,X-MC,16.99,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,0.53,-7.97,35.27,24.48,86780.0,-21319.0,246046.0,-9.53,38.0,X-LC,1.65,37.0,-0.25,1.75,42.62,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,1.31,-1.81,22.99,20.77,52857.0,-4232.0,229912.0,-10.33,35.0,X-MC,1.84,70.0,-0.08,1.64,4.99,X40N,NTT,BREWERIES
73,TCS,4389.97,-2.22,-12.14,37.39,20.71,124251.0,-45926.0,332311.0,-24.27,46.0,X-LC,2.12,3.0,-0.37,2.37,11.04,X40,ATH,IT
20,DABUR,735.00,-1.58,0.45,43.05,43.70,107278.0,1125.0,249193.0,-7.78,54.0,X-MC,2.16,73.0,0.01,1.78,15.57,XY24,BTT,FMCG
65,SIEMENS,4671.50,0.77,-20.19,57.27,25.51,85055.0,-37580.0,148515.0,-8.78,37.0,H-LC,2.35,50.0,-0.44,1.06,9.44,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.48,-29.11,121.73,57.19,281437.0,-94929.0,231198.0,-66.44,21.0,X-MC,10.46,58.0,-0.34,1.65,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.30,-41.61,131.55,35.20,98836.0,-53538.0,75132.0,-1.52,26.0,X-SC,17.59,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.43,-22.42,69.18,31.26,123860.0,-51728.0,179040.0,-63.74,26.0,X-MC,20.76,56.0,-0.42,1.28,0.0,X40N,ATH,IT
43,ITC,452.00,-0.09,-18.04,35.17,10.78,69272.0,-43356.0,196962.0,-51.37,13.0,X-LC,8.48,1.0,-0.63,1.40,0.0,X40,NTT,FMCG
52,PAGEIND,51605.07,-1.14,-13.31,50.67,30.62,104127.0,-31554.0,205500.0,-38.32,29.0,X-MC,16.99,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.17,-28.17,98.87,42.84,46163.0,-18315.0,46691.0,-53.17,50.0,X-SC,25.29,83.0,-0.40,0.33,5.63,XY24,NTT,MISC
58,RELAXO,1176.00,-0.49,-49.01,198.70,52.31,147070.0,-71144.0,74016.0,-45.70,36.0,X-SC,7.91,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.30,-41.61,131.55,35.20,98836.0,-53538.0,75132.0,-1.52,26.0,X-SC,17.59,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.81,-1.66,26.17,24.08,32672.0,-2103.0,124847.0,-9.66,47.0,X-SC,5.80,91.0,-0.06,0.89,17.63,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.68,-17.74,74.23,43.32,99453.0,-28892.0,133980.0,-29.29,45.0,X-SC,8.15,90.0,-0.29,0.95,2.87,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.09,-18.04,35.17,10.78,69272.0,-43356.0,196962.0,-51.37,13.0,X-LC,8.48,1.0,-0.63,1.40,0.00,X40,NTT,FMCG
75,TMPV,600.00,0.16,-28.20,71.28,22.98,115608.0,-63709.0,162189.0,-25.52,38.0,X-LC,3.74,2.0,-0.55,1.16,2.01,XY24,NTT,AUTO
73,TCS,4389.97,-2.22,-12.14,37.39,20.71,124251.0,-45926.0,332311.0,-24.27,46.0,X-LC,2.12,3.0,-0.37,2.37,11.04,X40,ATH,IT
81,VBL,671.64,0.32,1.57,33.58,35.67,107720.0,4944.0,320786.0,-10.45,61.0,X-LC,2.61,5.0,0.05,2.29,15.64,X40N,ATH,FMCG
41,INFY,1972.00,-0.66,8.13,24.15,34.24,81706.0,25441.0,338329.0,-15.65,44.0,X-LC,7.54,6.0,0.31,2.41,17.09,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,3.88,-30.17,90.31,32.89,48689.0,-23294.0,53913.0,-765.75,73.0,L-MC,8.62,259.0,-0.48,0.38,48.47,XR,NTT,BANKS
13,BSOFT,831.70,-2.09,-24.54,99.83,50.79,105951.0,-34522.0,106131.0,-3.36,46.0,H-SC,10.13,171.0,-0.33,0.76,23.57,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.89,-14.76,17.33,0.02,25787.0,-25759.0,148800.0,127.12,52.0,M-SC,8.79,240.0,-1.00,1.06,29.43,OX40N,NTT,PAINTS
17,COALINDIA,484.83,1.38,6.92,11.51,19.23,18517.0,10416.0,160876.0,25.73,71.0,L-LC,8.68,182.0,0.56,1.15,29.67,XY25,ATH,MINING
82,VOLTAS,1856.00,-1.88,12.13,29.49,45.20,63402.0,23253.0,214995.0,1.13,53.0,X-MC,5.56,80.0,0.37,1.53,19.75,XY25,NTT,AC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.38,6.92,11.51,19.23,18517.0,10416.0,160876.0,25.73,71.0,L-LC,8.68,182.0,0.56,1.15,29.67,XY25,ATH,MINING
0,5PAISA,593.00,-0.01,-32.11,66.29,12.90,98339.0,-70150.0,148346.0,118.75,54.0,H-SC,9.24,173.0,-0.71,1.06,23.35,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,3.88,-30.17,90.31,32.89,48689.0,-23294.0,53913.0,-765.75,73.0,L-MC,8.62,259.0,-0.48,0.38,48.47,XR,NTT,BANKS
72,TATAELXSI,9161.00,1.75,-16.51,58.14,32.03,63993.0,-21768.0,110067.0,-9.66,67.0,H-SC,6.69,157.0,-0.34,0.78,23.28,OX40N,NTT,IT
57,RECLTD,446.00,0.93,0.24,20.44,20.74,41629.0,495.0,203665.0,31.15,56.0,M-MC,5.23,189.0,0.01,1.45,10.88,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.59
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.68
MC    30.61
LC    26.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.40
X40      23.95
X40N     12.86
XY25     10.82
XR        8.91
AR        8.46
OX40N     7.64
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.23
X-LC    23.21
H-SC    22.81
M-SC    11.22
X-SC     7.89
H-MC     4.55
M-MC     1.45
M-LC     1.29
L-LC     1.15
H-LC     1.06
L-SC     0.76
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.26,-8.78,44.47
IT,12.77,-19.39,79.61
FINANCE,11.05,-14.88,63.10
MISC,7.03,-31.53,85.33
PAINTS,5.89,-11.68,28.27
ELECTRICAL,5.86,-14.22,56.06
INSURANCE,4.69,-2.87,38.08
PHARMA,3.94,-6.30,39.76
AUTO,2.80,-33.86,80.27


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3266482.0,21
AR,1386555.0,10
XR,1240745.0,12
X40,1158978.0,15
X40N,939431.0,9
OX40N,717490.0,10
XY25,424422.0,7
SR,285337.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3590105.0,24
X-MC,1634715.0,16
M-SC,1485812.0,15
X-LC,967549.0,13
X-SC,812401.0,8
H-MC,440402.0,3
L-SC,183363.0,2
M-LC,111203.0,1
H-LC,85055.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253012.0      6
           AR         938459.0      5
M-SC       XY24       852686.0      6
H-SC       XR         800272.0      7
X-MC       X40        537014.0      7
           XY24       452546.0      3
X-LC       X40        428119.0      6
X-MC       X40N       419106.0      4
X-SC       X40N       321510.0      3
H-SC       OX40N      313025.0      4
M-SC       OX40N      310920.0      5
X-SC       XY24       297046.0      3
H-SC       SR         285337.0      2
H-MC       AR         231598.0      2
X-MC       XY25       226049.0      2
H-MC       XY24       208804.0      1
X-LC       XY24       202388.0      2
           X40N       198815.0      2
X-SC       X40        193845.0      2
M-SC       XR         190763.0      2
X-LC       XY25       138227.0      3
M-SC       AR         131443.0      2
M-LC       XR         111203.0      1
L-SC       OX40N       93545.0      1
           XR          89818.0      1
H-LC       AR          85055.0      1
L-MC       XR          48689.0      1
M-MC       XY25        41629.0      1
L-LC       XY25        18517.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
